In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(1)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
#TEST NOTES:  Replacing batchnorm with instance norm; Adding "shock absorbing" training sessions between size changes.
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
OPENIMAGES = Path('data/openimages')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'super_res'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

#gpath = IMAGENET.parent/('bwc_withattn_sn_supertrain2x_gen_96.h5')
#dpath = IMAGENET.parent/('bwc_withattn_sn_supertrain2x_critic_96.h5')

c_lr=1e-3
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/5
g_lrs = np.array([g_lr/100,g_lr/10,g_lr])

keep_pcts=[0.20,0.20]
gen_freeze_tos=[-1,0]

scale=2
sn=True
self_attention=sn
lrs_unfreeze_factor=0.25
x_tfms=[]
extra_aug_tfms=[]

torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=1, self_attention=self_attention, sn=sn, leakyReLu=False, scale=scale).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=128, scale=16, self_attention=self_attention, sn=sn).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
trainer = WGANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e2)], sn=sn)
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=100)

In [6]:
scheds=[]

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[128,128], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos, reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[128,128], bss=[32,32], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3,0.3], 
    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos, reduce_x_scale=scale))



#unshock
scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/20, g_lrs=g_lrs/20, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/6, g_lrs=g_lrs/6, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/6, g_lrs=g_lrs/6, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))


#unshock
scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/120, g_lrs=g_lrs/120, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/12, g_lrs=g_lrs/12, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/12, g_lrs=g_lrs/12, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/24, g_lrs=g_lrs/24, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/24, g_lrs=g_lrs/24, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))



In [ ]:
trainer.train(scheds=scheds)

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


Failed to generate graph for model: %43 : Long() = onnx::Constant[value={0}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400


/media/jason/Projects/Deep Learning/Deep Learning Projects/super resolution/fasterai/generators.py:161: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h<target_h or w<target_w:


Failed to generate graph for model: %479 : Long() = onnx::Constant[value={2}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400
  0%|          | 0/7627 [00:00<?, ?it/s]

/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


  0%|          | 20/7627 [00:29<1:47:17,  1.18it/s]
WDist 1.8324944972991943; RScore 1.2281556129455566; FScore 0.6043388843536377; GAddlLoss [7.97772]; Iters: 10; GCost: 2.2304227352142334; GPenalty: [0]; ConPenalty: [0]
  1%|          | 40/7627 [00:41<54:57,  2.30it/s]  
WDist 1.8389149904251099; RScore 1.771890640258789; FScore 0.06702432036399841; GAddlLoss [7.09086]; Iters: 20; GCost: -0.031806934624910355; GPenalty: [0]; ConPenalty: [0]
  1%|          | 60/7627 [00:56<43:00,  2.93it/s]
WDist 1.6337389945983887; RScore 0.6577204465866089; FScore 0.9760186076164246; GAddlLoss [6.93698]; Iters: 30; GCost: 0.15023335814476013; GPenalty: [0]; ConPenalty: [0]
  1%|          | 80/7627 [01:11<37:48,  3.33it/s]
WDist 1.6346361637115479; RScore 0.4791252017021179; FScore 1.1555110216140747; GAddlLoss [6.57196]; Iters: 40; GCost: 0.040726568549871445; GPenalty: [0]; ConPenalty: [0]
